# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 19th 2025

**Student Name**: Luis Adrian Bravo Ramirez

**Professor**: Pablo Camarillo Ramirez

# Find the PySpark Installation & Create SparkSession

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://4f3c6067fdf1:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/19 15:02:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Define the Schema & Load CSV

In [3]:
from pcamarillor.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

In [4]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/airline/")

df_airlines.show(n=10)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
|    5| Vistara| UK-945|

# Data Cleaning, Columns Creations and Transformation

- Drop unnecessary columns. Count how many null values dataset have before/after
cleaning process.

In [6]:
from pyspark.sql.functions import trim, col, count, isnull, when

# Get number of null values for each column before cleaning 
print(f"Number of records before cleaning: {df_airlines.count()}")
df_airlines.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

# Perform data cleaning with trim (column by column)
airlines_clean = df_airlines \
        .dropDuplicates(["index"]) \
        .withColumn("airline", trim("airline")) \
        .withColumn("source_city", trim("source_city")) \
        .withColumn("destination_city", trim("destination_city")) \
        .filter(col("price").isNotNull())

# Perform data cleaning with dropna (make sure that it erases null values)
airlines_clean = airlines_clean.dropna()

print(f"\nNumber of records after cleaning with trim & dropna: {airlines_clean.count()}\n")

airlines_clean.show(n=10)

Number of records before cleaning: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+




Number of records after cleaning with trim & dropna: 300153



+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+
|index|  airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|
+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+
|    1| SpiceJet|SG-8157|      Delhi| Early_Morning| zero|     Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    3|  Vistara| UK-995|      Delhi|       Morning| zero|   Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    5|  Vistara| UK-945|      Delhi|       Morning| zero|   Afternoon|          Mumbai|Economy|    2.33|        1| 5955|
|    6|  Vistara| UK-927|      Delhi|       Morning| zero|     Morning|          Mumbai|Economy|    2.08|        1| 6060|
|    9| GO_FIRST| G8-336|      Delhi|     Afternoon| zero|     Evening|          Mumbai|Economy|    2.25|        1| 5954|
|   12|   Indigo|6E-5001

- Normalize categorical values: map ”zero” → 0, ”one” → 1, etc. in stops.

In [7]:
from pyspark.sql.functions import when, lit

airlines_clean = airlines_clean.withColumn(
    "stops",
    when(col("stops") == "zero", lit(0))
    .when(col("stops") == "one", lit(1))
    .otherwise(lit(2))
)

airlines_clean.show(n=20)

airlines_clean.select("stops").distinct().show()


+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+
|index|  airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|
+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+
|    1| SpiceJet|SG-8157|      Delhi| Early_Morning|    0|     Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    3|  Vistara| UK-995|      Delhi|       Morning|    0|   Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    5|  Vistara| UK-945|      Delhi|       Morning|    0|   Afternoon|          Mumbai|Economy|    2.33|        1| 5955|
|    6|  Vistara| UK-927|      Delhi|       Morning|    0|     Morning|          Mumbai|Economy|    2.08|        1| 6060|
|    9| GO_FIRST| G8-336|      Delhi|     Afternoon|    0|     Evening|          Mumbai|Economy|    2.25|        1| 5954|
|   12|   Indigo|6E-5001

+-----+
|stops|
+-----+
|    1|
|    2|
|    0|
+-----+



- Create a new column called route: ”Delhi → Mumbai” from source city and
destination city.

In [8]:
from pyspark.sql.functions import concat_ws

airlines_clean =  airlines_clean.withColumn(
    "route",
    concat_ws(" -> ", col("source_city"), col("destination_city"))
)

airlines_clean.show(n=10)

airlines_clean.select("route").distinct().show(n=10)

+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+---------------+
|index|  airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|          route|
+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+---------------+
|    1| SpiceJet|SG-8157|      Delhi| Early_Morning|    0|     Morning|          Mumbai|Economy|    2.33|        1| 5953|Delhi -> Mumbai|
|    3|  Vistara| UK-995|      Delhi|       Morning|    0|   Afternoon|          Mumbai|Economy|    2.25|        1| 5955|Delhi -> Mumbai|
|    5|  Vistara| UK-945|      Delhi|       Morning|    0|   Afternoon|          Mumbai|Economy|    2.33|        1| 5955|Delhi -> Mumbai|
|    6|  Vistara| UK-927|      Delhi|       Morning|    0|     Morning|          Mumbai|Economy|    2.08|        1| 6060|Delhi -> Mumbai|
|    9| GO_FIRST| G8-336|      Del

+--------------------+
|               route|
+--------------------+
|    Delhi -> Chennai|
|  Hyderabad -> Delhi|
|    Chennai -> Delhi|
|   Mumbai -> Chennai|
|Hyderabad -> Kolkata|
| Hyderabad -> Mumbai|
| Mumbai -> Bangalore|
|    Delhi -> Kolkata|
|   Mumbai -> Kolkata|
|Bangalore -> Kolkata|
+--------------------+
only showing top 10 rows


- Transform departure time and arrival time to numerical category (Morning,
Afternoon, etc.), then encode as numbers (0=Early Morning, 1=Morning, etc.)

In [9]:
airlines_clean = airlines_clean.withColumn(
    "departure_time",
    when(col("departure_time") == "Early_Morning", 0)
    .when(col("departure_time") == "Morning", 1)
    .when(col("departure_time") == "Afternoon", 2)
    .when(col("departure_time") == "Evening", 3)
    .when(col("departure_time") == "Night", 4)
    .when(col("departure_time") == "Late_Night", 5)
).withColumn(
    "arrival_time",
    when(col("arrival_time") == "Early_Morning", 0)
    .when(col("arrival_time") == "Morning", 1)
    .when(col("arrival_time") == "Afternoon", 2)
    .when(col("arrival_time") == "Evening", 3)
    .when(col("arrival_time") == "Night", 4)
    .when(col("arrival_time") == "Late_Night", 5)
)

airlines_clean.show(n=10)

airlines_clean.select("departure_time").distinct().show()
airlines_clean.select("arrival_time").distinct().show()

+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+---------------+
|index|  airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|          route|
+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+---------------+
|    1| SpiceJet|SG-8157|      Delhi|             0|    0|           1|          Mumbai|Economy|    2.33|        1| 5953|Delhi -> Mumbai|
|    3|  Vistara| UK-995|      Delhi|             1|    0|           2|          Mumbai|Economy|    2.25|        1| 5955|Delhi -> Mumbai|
|    5|  Vistara| UK-945|      Delhi|             1|    0|           2|          Mumbai|Economy|    2.33|        1| 5955|Delhi -> Mumbai|
|    6|  Vistara| UK-927|      Delhi|             1|    0|           1|          Mumbai|Economy|    2.08|        1| 6060|Delhi -> Mumbai|
|    9| GO_FIRST| G8-336|      Del

+--------------+
|departure_time|
+--------------+
|             1|
|             3|
|             5|
|             4|
|             2|
|             0|
+--------------+



+------------+
|arrival_time|
+------------+
|           1|
|           3|
|           5|
|           4|
|           2|
|           0|
+------------+



- Add a new column is expensive: when(price > 6000, True).otherwise(False).

In [10]:
airlines_clean = airlines_clean.withColumn(
    "is_expensive",
    when(col("price") > 6000, lit(True)).otherwise(lit(False))
)

airlines_clean.show()

+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+---------------+------------+
|index|  airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|          route|is_expensive|
+-----+---------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+---------------+------------+
|    1| SpiceJet|SG-8157|      Delhi|             0|    0|           1|          Mumbai|Economy|    2.33|        1| 5953|Delhi -> Mumbai|       false|
|    3|  Vistara| UK-995|      Delhi|             1|    0|           2|          Mumbai|Economy|    2.25|        1| 5955|Delhi -> Mumbai|       false|
|    5|  Vistara| UK-945|      Delhi|             1|    0|           2|          Mumbai|Economy|    2.33|        1| 5955|Delhi -> Mumbai|       false|
|    6|  Vistara| UK-927|      Delhi|             1|    0|           1|          Mumbai|Econom

# Aggregations

- Get the average price per airline

In [15]:
from pyspark.sql.functions import col, avg

airlines_clean.groupBy(col('airline')).agg(avg(col('price')).alias("Average Price")).show()

+---------+------------------+
|  airline|     Average Price|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
|  Vistara| 30396.53630170735|
+---------+------------------+



- Average duration per route

In [18]:
airlines_clean.groupBy(col('route')).agg(avg(col('duration')).alias("Average Duration")).show()

+--------------------+------------------+
|               route|  Average Duration|
+--------------------+------------------+
|    Delhi -> Chennai|12.433964745763944|
|  Hyderabad -> Delhi|10.829816602522587|
|    Chennai -> Delhi|  11.1493744312541|
|   Mumbai -> Chennai|12.665900287564627|
|Hyderabad -> Kolkata|13.535322410033165|
| Hyderabad -> Mumbai|11.962923295795918|
| Mumbai -> Bangalore|11.612022516178817|
|    Delhi -> Kolkata| 12.73596614766045|
|   Mumbai -> Kolkata|12.836848115489666|
|Bangalore -> Kolkata|13.099143404859825|
| Mumbai -> Hyderabad|13.263310412247066|
|    Kolkata -> Delhi| 11.60498857561711|
|Hyderabad -> Chennai|13.293238468912078|
|  Chennai -> Kolkata|14.515774035955694|
|     Delhi -> Mumbai|10.367774213738123|
|   Kolkata -> Mumbai|12.991932481150478|
|     Mumbai -> Delhi|  9.81805726844943|
|Kolkata -> Hyderabad|13.853107514948396|
|Chennai -> Bangalore|13.952593563812163|
|   Chennai -> Mumbai|12.374656244132625|
+--------------------+------------

- Minimum and maximum price per airline

In [19]:
from pyspark.sql.functions import min, max

airlines_clean.groupBy(col('airline')).agg(
    min('price').alias('Min Price'),
    max('price').alias('Max Price')
).show()

+---------+---------+---------+
|  airline|Min Price|Max Price|
+---------+---------+---------+
|   Indigo|     1105|    31952|
| SpiceJet|     1106|    34158|
|Air_India|     1526|    90970|
|  AirAsia|     1105|    31917|
| GO_FIRST|     1105|    32803|
|  Vistara|     1714|   123071|
+---------+---------+---------+



- Count flights by departure time category

In [22]:
airlines_clean.groupBy(col('departure_time')).count().orderBy(col('count')).show()

+--------------+-----+
|departure_time|count|
+--------------+-----+
|             5| 1306|
|             2|47794|
|             4|48015|
|             3|65102|
|             0|66790|
|             1|71146|
+--------------+-----+

